In [ ]:
#DERS-30

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np

In [ ]:
data = pd.read_csv("winequality-red.csv")
df = data.copy()
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
sns.pairplot(df)
plt.show()

In [ ]:
sns.heatmap(df.corr(),annot=True,cbar=True)
plt.show()

In [ ]:
#Principal Component Analysis (PCA) (Temel Bileşenler Analizi) KURULUMU
y = df["quality"]
X = df.drop("quality",axis=1)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


#Elimizdeki bağımsız değişken yapılarını standartlaştırma işlemi yapıyoruz
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#PCA işlemlerine geçiyoruz, yani verileri küçültüyoruz
pca = PCA() #n_components=2 = 11 birleşenimi 2 birleşene indirmek istiyorum

X_train2 = pca.fit_transform(X_train)
X_test2 = pca.transform(X_test)

print(X_train.shape)
print(X_train2.shape)

In [ ]:
print(np.cumsum(pca.explained_variance_ratio_)*100)#Açıklanan varyans oranlarını bana kümülatif bir şekilde verecek
#Eğer 8 tane değişken ile çalışırsam, senin açıklanan varyans oranın %94.70160536 diyor bize
#Elindekini minimize etmeye çalışırken %90 üzerinde açıklanan varyans oranına da sahip olacağız

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel("Bileşen Sayısı")
plt.ylabel("Açıklanan Varyans")
plt.show()

In [ ]:
#DERS-31

In [ ]:
from sklearn.linear_model import LinearRegression
import sklearn.metrics as mt

In [ ]:
lm = LinearRegression()
lm.fit(X_train2,y_train)
tahmin = lm.predict(X_test2)

r2 = mt.r2_score(y_test,tahmin)
mse = mt.mean_squared_error(y_test,tahmin,squared=True)

print(f"R2: {r2}, MSE: {mse}")

In [ ]:
#Çapraz doğrulama işlemiyle veri setini parçalara ayıracak ve bunlara karşılık gelen her bir yapıya cross_val_score alacak
#Alınan bu cross_val_score skorunun ortalamasını alıp kaç bileşen ile çalışabileceğimi gösteren bir yapı kuracağız
#shuffle = her seçme yapısında karıştırma işlemi gerçekleştiriyor
cv = KFold(n_splits=10,shuffle=True,random_state=1)

lm2 = LinearRegression()
RMSE = []
#X_train2.shape[1] = Bu değer 11 anlamına geliyor, +1 de dahil, dahil değil muhabbeti
for i in range(1,X_train2.shape[1]+1):
    hata = np.sqrt(-1*cross_val_score(lm2,X_train2[:,:i],
                                      y_train.ravel(),
                                      cv=cv,
                                      scoring="neg_mean_squared_error").mean())
    RMSE.append(hata)
    #Scoring bize mean_squared_error negatif halini gönderecek, biz de -1 ile çarpıp pozitif haline getireceğiz
    #Artıya dönen sonucun karekökünü alarak RMSE'ye çevirmiş olacağız
#-x çizginin kırılımlarını gösteriyor
plt.plot(RMSE,"-x")
plt.xlabel("Bileşen Sayısı")
plt.ylabel("RMSE")
#Aslında tam kırılımı bulmak lazım, 3. den sonra aslında bir düzlüğe çıkmış oluyor